# USING OLLAMA LOCALLY

In [ ]:
import os
import pprint
import shutil 
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings

# 1. Load your preprocessed training data

loader = TextLoader("Train.txt", autodetect_encoding=True)
docs = loader.load()
print(f"✅ Loaded {len(docs)} documents")
pprint.pprint(docs[0].metadata)

# 2. Split into chunks
 
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
chunks = splitter.split_documents(docs)
 
# 3. Create embeddings using HuggingFace
 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("🔍 Embedding size:", len(embeddings.embed_query("test")))
 
# 4. Setup persistent vector store (Chroma)
 
db_dir = os.path.join(os.getcwd(), "db")

def create_vector_store(docs, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    
    
    if os.path.exists(persistent_directory):
        print(f"⚠️ Deleting existing vector store: {store_name}")
        shutil.rmtree(persistent_directory)
    
    print(f"\n--- Creating vector store: {store_name} ---")
    Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory
    ).persist()
    print("--- ✅ Vector store created successfully ---")

# 5. Load vector store and create retriever
 
persistent_directory = os.path.join(db_dir, "loan_vector_store")
db = Chroma(
    persist_directory=persistent_directory,
    embedding_function=embeddings,
)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
 
# 6. Define custom prompt
 
prompt_template = """
You are a helpful assistant trained on a dataset of past loan applications.
Use only the provided context to answer the user's query.
If the answer is not found in the context, say "I'm not sure."

Context:
{context}

Question:
{question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
 
# 7. Load Ollama Local LLM 
 
llm = ChatOllama(
    model="llama3", 
    temperature=0,
)
 
# 8. Create QA chain with retriever + prompt
 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
 
# 9. Ask a question
 
query = "What kind of applicants usually get loans approved?"
result = qa_chain.invoke({"query": query})
 
# 10. Display result and source
 
print("\n--- ✅ Answer ---")
print(result["result"])

print("\n--- 🧾 Source Documents ---")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n📄 Document {i}:")
    print(doc.page_content)
    print("—" * 50)


✅ Loaded 1 documents
{'source': 'Train.txt'}


c:\Users\acer\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔍 Embedding size: 384


C:\Users\acer\AppData\Local\Temp\ipykernel_24744\860026814.py:55: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
C:\Users\acer\AppData\Local\Temp\ipykernel_24744\860026814.py:86: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(



--- ✅ Answer ---
Based on our training data, we've seen that applicants with a stable income, good credit history, and a reasonable debt-to-income ratio tend to have their loan applications approved more frequently. Additionally, applicants who provide thorough documentation and have a clear plan for using the loan funds are also more likely to get approval.

--- 🧾 Source Documents ---


# Testing on queries regarding Loan Approval (according to the dataset)

In [2]:
queries = [
    "What are common reasons for loan rejections?",
    "How important is the credit score in loan approval?",
    "Do employment status affect loan approval?"
]
for q in queries:
    print(f"\n🔍 Query: {q}")
    result = qa_chain.invoke({"query": q})
    print("Answer:", result["result"])



🔍 Query: What are common reasons for loan rejections?
Answer: Based on our training data, we've identified some common reasons for loan rejections. These include:

* Insufficient credit history or poor credit score
* High debt-to-income ratio
* Inconsistent or incomplete application information
* Unverifiable income or employment status
* Lack of collateral or insufficient asset value
* High loan-to-value (LTV) ratio

These are just a few examples, but it's not an exhaustive list. If you have any specific questions or concerns about your own loan application, I'd be happy to help!

🔍 Query: How important is the credit score in loan approval?
Answer: Based on our training data, we can see that credit score plays a significant role in loan approval. In fact, it's one of the most important factors considered by lenders when evaluating loan applications. A good credit score (typically above 700) indicates to lenders that you're likely to repay your debt on time and in full, which reduces 

In [3]:
queries = [
    "I have a salary of ₹50,000 per month, no existing loans, and a credit score of 780. Will my loan get approved?",
    "I'm self-employed with an irregular income and a credit score of 650. Do I have a chance for approval?",
    "My CIBIL score is 580 but I have a permanent government job. Can I still get a loan?",
    "I already have a personal loan and a home loan. Will I get another loan for my car?",
    "I'm 22 years old, just started working, and have no credit history. Will my loan be accepted?",
    "What are the most important factors that determine loan approval?",
    "Does age or marital status affect loan eligibility?",
    "How much monthly income do I need to get a ₹5 lakh loan approved?",
    "Is a co-applicant necessary for people with low income?",
    "Can students apply for loans without collateral?",
    "What is the minimum credit score required to get a loan approved?",
    "My credit score dropped recently due to a missed EMI. Will that affect my application?",
    "Can I get a loan if my credit score is good but I recently changed jobs?",
    "How can I improve my chances of getting a personal loan approved?",
    "Should I reduce my credit card utilization before applying for a loan?",
    "Is it better to apply for a loan with a bank or NBFC?",
    "My loan application was rejected. Can I reapply with a co-signer?",
    "What documents are usually missing that cause a rejection?",
    "If my loan is rejected once, does it affect future applications?",
    "What documents should I submit for a smooth loan approval process?",
    "Does submitting more documents improve approval chances?",
    "How long does the loan approval process usually take?",
    "If I add my spouse as co-applicant, will that improve approval odds?",
    "I have two job offers. Should I wait to join the better-paying one before applying?",
    "Which gets faster approval — a salaried or self-employed applicant?"
]

for q in queries:
    print(f"\n🔍 Query: {q}")
    result = qa_chain.invoke({"query": q})
    print("Answer:", result["result"])



🔍 Query: I have a salary of ₹50,000 per month, no existing loans, and a credit score of 780. Will my loan get approved?
Answer: Based on the provided context, I can tell you that your credit score is excellent (780), which is a strong indicator for loan approval. Additionally, having no existing loans suggests that you have a good track record of managing debt.

Given these factors, it's likely that your loan application will get approved. However, please note that loan approvals also depend on other factors such as the loan amount, tenure, and purpose of the loan. But with an excellent credit score and no existing loans, you're off to a great start!

🔍 Query: I'm self-employed with an irregular income and a credit score of 650. Do I have a chance for approval?
Answer: Based on our dataset of past loan applications, we've seen that applicants with irregular incomes often face challenges when applying for loans. However, it's not impossible to get approved.

In your case, having a cred